In [2]:
import os

# Đường dẫn đến thư mục chứa các file txt
folder_path = "../doc_RAG"

# Hàm để đọc nội dung của một file txt và trả về từ điển
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    file_name = os.path.basename(file_path)
    # Lấy phần tiêu đề của file (không bao gồm phần đuôi .txt)
    title = os.path.splitext(file_name)[0]
    return {"title": title, "content": content}

# Lưu trữ các biến từ điển vào một list
file_data_list = []

# Lấy danh sách các file trong thư mục
files = os.listdir(folder_path)

# Đọc từng file và thêm dữ liệu vào list
for file in files:
    if file.endswith('.txt'):  # Chỉ xử lý các file có đuôi là .txt
        file_path = os.path.join(folder_path, file)
        file_data = read_txt_file(file_path)
        file_data_list.append(file_data)

# In ra danh sách các biến từ điển đã được tạo
for data in file_data_list:
    print(data)


{'title': 'NVIDIA_Agreements', 'content': 'AGREEMENTS by Entrant\n\n1, By entering this Contest, each entrant agrees and acknowledges that Sponsor and Sponsor Related Parties shall not be responsible or liable for: (i) any late, lost, stolen, forged, mutilated, corrupted, incomplete, illegible or misdirected entries; (ii) any errors, omissions, misinformation, or misidentification in an entry; (iii) any dispute or claim arising from an entrant’s participation in this Contest or his/her entry, or his/her receipt, ownership or use of the prize; (iv) any computer hardware, software, Internet, network, cable, phone, or other communication or technical errors, failures, malfunctions, interruptions, or delays; or (v) any damage to an entrant’s or any other person’s computer hardware, software or data that results from participation in this Contest or accessing, downloading or using any tools, files, data, software, or other articles or materials in connection with this Contest.\n2, By enteri

In [3]:
import pandas as pd

df = pd.DataFrame(file_data_list)
df.columns= ['Title', 'Text']
df

C:\Users\cungh\AppData\Local\Temp\ipykernel_10624\2810620536.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Title,Text
0,NVIDIA_Agreements,"AGREEMENTS by Entrant\n\n1, By entering this C..."
1,NVIDIA_contest_information,Contest Information\nThe Sponsor of this promo...
2,NVIDIA_Eligibility,Eligibility\nYou must be a resident of Argenti...
3,NVIDIA_Entry_Period,Entry Period\nThe entry period for this Contes...
4,NVIDIA_Governing,Governing Law; Jurisdiction\nThese Official Ru...
5,NVIDIA_introduction,NVIDIA Gen AI on RTX PCs Developer Contest\nBu...
6,NVIDIA_intro_term_and_conditions,Contest Terms and Conditions\nBy entering this...
7,NVIDIA_Privacy,Privacy\nBy entering this promotion you expres...
8,NVIDIA_Prizes_and_Odds_of_Winning,Prizes and Odds of Winning\nThe prizes of this...
9,NVIDIA_Reservation,Reservation of Rights by SPONSOR\nSponsor rese...


In [4]:
# load E5-small
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('../model/E5')
model = AutoModel.from_pretrained('../model/E5')

d:\NVIDIA_contest\NVIDIA_CONTEST\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
list_Text = list(df['Text'])


In [6]:
batch_dict = tokenizer(list_Text, max_length=512, padding=True, truncation=True, return_tensors='pt')
outputs = model(**batch_dict)

In [7]:
from torch import Tensor

def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

In [8]:
list_embeddings = list(embeddings)

In [9]:
# Chuyển đổi các tensor thành mảng NumPy
list_numpy_arrays_embeddings = [tensor.detach().numpy() for tensor in list_embeddings]


In [10]:
df['Embeddings'] = list_numpy_arrays_embeddings

In [11]:
print(df)

                                Title  \
0                   NVIDIA_Agreements   
1          NVIDIA_contest_information   
2                  NVIDIA_Eligibility   
3                 NVIDIA_Entry_Period   
4                    NVIDIA_Governing   
5                 NVIDIA_introduction   
6    NVIDIA_intro_term_and_conditions   
7                      NVIDIA_Privacy   
8   NVIDIA_Prizes_and_Odds_of_Winning   
9                  NVIDIA_Reservation   
10                     NVIDIA_Sponsor   
11                       NVIDIA_Taxes   
12               NVIDIA_the_Challenge   
13         NVIDIA_Winner_Announcement   

                                                 Text  \
0   AGREEMENTS by Entrant\n\n1, By entering this C...   
1   Contest Information\nThe Sponsor of this promo...   
2   Eligibility\nYou must be a resident of Argenti...   
3   Entry Period\nThe entry period for this Contes...   
4   Governing Law; Jurisdiction\nThese Official Ru...   
5   NVIDIA Gen AI on RTX PCs Developer Con

In [12]:
# Chuyển đổi cột chứa numpy.ndarray thành list
df['Embeddings'] = df['Embeddings'].apply(lambda x: x.tolist())

In [13]:
df

,Title,Text,Embeddings
0,NVIDIA_Agreements,"AGREEMENTS by Entrant\n\n1, By entering this C...","[0.019041230902075768, -0.13747310638427734, -..."
1,NVIDIA_contest_information,Contest Information\nThe Sponsor of this promo...,"[0.017585821449756622, 0.00943501852452755, -0..."
2,NVIDIA_Eligibility,Eligibility\nYou must be a resident of Argenti...,"[-0.022351747378706932, 0.029725857079029083, ..."
3,NVIDIA_Entry_Period,Entry Period\nThe entry period for this Contes...,"[0.005828686524182558, -0.20511609315872192, -..."
4,NVIDIA_Governing,Governing Law; Jurisdiction\nThese Official Ru...,"[0.1846667230129242, 0.029188424348831177, -0...."
5,NVIDIA_introduction,NVIDIA Gen AI on RTX PCs Developer Contest\nBu...,"[0.1351611167192459, -0.2027195543050766, -0.2..."
6,NVIDIA_intro_term_and_conditions,Contest Terms and Conditions\nBy entering this...,"[0.2492372840642929, -0.07882043719291687, -0...."
7,NVIDIA_Privacy,Privacy\nBy entering this promotion you expres...,"[0.10994419455528259, -0.12684088945388794, -0..."
8,NVIDIA_Prizes_and_Odds_of_Winning,Prizes and Odds of Winning\nThe prizes of this...,"[0.03309042379260063, -0.1956143081188202, -0...."
9,NVIDIA_Reservation,Reservation of Rights by SPONSOR\nSponsor rese...,"[0.18865609169006348, -0.09743929654359818, -0..."


In [14]:
# Lưu DataFrame thành file CSV
df.to_csv('Embeddings_database.csv', index=False)